<a href="https://colab.research.google.com/github/JaejinLee1215/DeepLearning/blob/main/JaejinLee1215/ch09_DL_15_KoGPT_%EC%9D%B4%EC%9E%AC%EC%A7%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 챗봇 Dataset

* https://wikidocs.net/book/7439
* https://wikidocs.net/157896

* GPT-2는 주어진 텍스트의 다음 단어를 잘 예측할 수 있도록 학습된 언어모델이며 문장 생성에 최적화 되어 있습니다.
* KoGPT2는 부족한 한국어 성능을 극복하기 위해 40GB 이상의 텍스트로 학습된 한국어 디코더(decoder) 언어모델입니다.

* https://github.com/haven-jeon/KoGPT2-chatbot


In [1]:
!pip install transformers -q # colab에는 transformers 설치가 안 되어 있음
# pytorch나 keras 등 상관없이 transformer 기반의 신경망 알고리즘의 전이학습을 위한 패키지
# https://huggingface.co/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.6 MB/s eta 0:00:00


In [2]:
import math
import numpy as np
import pandas as pd
import random
import re
import torch

from torch.utils.data import DataLoader, Dataset
from transformers import PreTrainedTokenizerFast # 토큰화를 위한 라이브러리 (토크나이저)

한국어 챗봇 데이터는 https://github.com/songys/Chatbot_data 송영숙님 데이터를 사용합니다.

송영숙님의 챗봇 데이터 (https://github.com/songys/Chatbot_data) 의 구조를 그대로 사용하여 데이터의 Q 필드를 발화, A 필드를 발화 그리고 감정 레이블을 사용합니다. 감정 레이블은 label에 정의된 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2)를 그대로 적용합니다.

데이터의 Q 필드를 발화, A 필드를 발화 그리고 감정 레이블을 로 매핑해 P(|, )를 최대화 할 수 있는 모델을 학습합니다. 감정 레이블은 이곳의 정의를 따른다(일상다반사 0, 이별(부정) 1, 사랑(긍정) 2).

챗봇 데이터를 사용하여 Dataset, DataLoader 기능을 테스트 해 봅니다.

먼저 필요한 라이브러리를 import 합니다.

In [3]:
Chatbot_Data = pd.read_csv("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv")

# Test 용으로 300개 데이터만 처리한다.
Chatbot_Data = Chatbot_Data[:300]
Chatbot_Data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


Tokenizer는 모델에 어떠한 입력을 넣어주기 위해서 전처리를 담당합니다. 토크나이저는 허깅페이스의 PreTrainedTokenizer 인 GPT2Tokenizer 를 사용합니다. Tokenizer들은 크게 3가지 기능을 제공합니다.

1. Tokenizing : 입력 문자열을 token id로 변환(encoding), token id를 다시 문자열로 변환(decoding)의 기능
1. 기존의 구조(BPE, Sentencepiece 등)에 독립적으로 추가적인 token들을 추가하는 기능
1. Special token들을 (mask, BOS, EOS 등) 관리하는 기능


In [4]:
BOS = "</s>" # Begining of Speech
EOS = "</s>" # End of speech
PAD = "<pad>"
MASK = "<unused0>"
U_TKN = '<usr>' # 유저가 입력해준 문장
S_TKN = '<sys>' # 시스템(챗봇)이 답변하는 문장
SENT = '<unused1>'

# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
# https://huggingface.co/skt/kogpt2-base-v2
# https://github.com/SKT-AI/KoGPT2
# https://github.com/seawavve/newsTopicClassification/blob/main/NewsTopicClassificationwithKoGPT2.ipynb
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


사용된 파라미터의 의미는 다음과 같습니다.

* bos_token : 문장의 시작을 나타내는 token
* eos_token : 문장의 끝을 나타내는 token
* unk_token : 모르는 단어를 나타내는 token
* pad_token : 동일한 batch 내에서 입력의 크기를 동일하게 하기 위해서 사용해는 token

PreTrainedTokenizer 에서 제공되는 함수는

* tokenize() : tokenizer를 이용해서 string을 token id의 리스트로 변환한다.
* get_added_vocab() : token to index에 해당하는 dict를 리턴한다.
* batch_decode() : token id로 구성된 입력을 하나의 연결된 string으로 출력한다.
* convert_ids_to_tokens() : token id 의 리스트를 token으로 변환한다.
* skip_special_tokens=True로 하면 decoding할 때 special token들을 제거한다.
* convert_tokens_to_ids() : token string의 리스트를 token id 또는 Token id의 리스트로 변환한다.
* decode() : tokenizer 와 vocabulary를 이용해서 token id를 string으로 변환한다. skip_special_token=True로 지정하면 speical token들을 제외한다.
* encode() : token string을 token id 의 리스트로 변환한다. add_special_tokens=False로 지정하면 token id로 변환할 때 special token들을 제외한다. padding을 통해서 padding token을 어떻게 추가할지조 지정할 수 있다.

이제 Dataset을 정의 합니다.

In [5]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        # self.q_token = Q_TKN
        # self.a_token = A_TKN
        self.q_token = U_TKN
        self.a_token = S_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

리턴되는 데이터는 token_ids, mask, labels_ids 입니다. token_ids 는 + 질문문장 + + 감정 + + 답변 + + pad_token_id 순서 입니다. pad_token_id는 max_len 에 일치하도록 추가 됩니다. mask 는 질문 q가 들어 가는 곳에는 0, 답변 a가 위치한 곳에는 1 그리고 빈 공간에는 0 으로 채워 집니다. labels은 질문의 길이만큼 mask 문자 그리고 답변 a의 id 입니다.

배치 데이터를 만들기 위해 collate_batch 함수를 정의합니다.

In [6]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [7]:
train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=2, shuffle=True, collate_fn=collate_batch,)

In [8]:
print("start")
for batch_idx, samples in enumerate(train_dataloader):
    token_ids, mask, label = samples
    print("token_ids ====> ", token_ids)
    print("mask =====> ", mask)
    print("label =====> ", label)
print("end")

start


<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


token_ids ====>  tensor([[    2,   739,  6910,  ...,     3,     3,     3],
        [    2, 25883, 11630,  ...,     3,     3,     3],
        [    2,  9067,  7653,  ...,     3,     3,     3],
        ...,
        [    2,  9244,  6958,  ...,     3,     3,     3],
        [    2,  9815, 37655,  ...,     3,     3,     3],
        [    2, 25883, 11630,  ...,     3,     3,     3]])
mask =====>  tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
label =====>  tensor([[9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3],
        [9, 9, 9,  ..., 3, 3, 3]])
token_ids ====>  tensor([[    2, 11457, 10051,  ...,     3,     3,     3],
        [    2,  9244, 16905,  ...,     3,     3,     3],
        [    2, 19319, 46

## koGPT2 ChatBot

Torch와 GPT2LMHeadModel 을 import 합니다.

In [9]:
import torch
from transformers import GPT2LMHeadModel

SK텔레콤이 공개한 KoGPT2 모델이 사용하는 토크나이저를 선언합니다. eos_token은 문장 마지막에 붙이는 스페셜 토큰(end of sentence)으로 SK텔레콤이 모델을 프리트레인할 때 이렇게 지정했기 때문에 우리도 같은 방식으로 사용합니다.

In [10]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>') 
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [11]:
#사전학습된 koGPT2 언어 모델을 불러 옵니다.

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

In [12]:
# 모델에 Seed Text를 넣고 문장생성을 테스트 해 봅니다.

text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)
# https://huggingface.co/spaces/gogamza/kogpt2-base-v2

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


In [13]:
# koGPT2 챗봇
# 실제 모델을 학습 시키는 코드를 작성해 보겠습니다. 먼저 필요한 라이브러리들을 import 합니다.
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
import re

In [14]:
# 토크나이저에서 사용되는 스페셜 토큰들을 정의 합니다.
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

In [15]:
# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 모델과 토크나이저를 가져옵니다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            bos_token=BOS, eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Chatbot_Data = pd.read_csv("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv")
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=2, shuffle=True, collate_fn=collate_batch,)

In [17]:
device

device(type='cuda')

In [18]:
# 모델을 추가로 학습하기 위해 .train() 사용합니다.
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [19]:
# 학습에 필요한 하이퍼 파라메터들을 선언 합니다.
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18

In [20]:
from tqdm.notebook import tqdm

# 학습
print("start")
for epoch in range(epoch):
    for batch_idx, samples in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids.to(device))
        out = out.logits.to(device)      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2).to(device)
        mask_out = torch.where(mask_3d == 1, out.to(device), Sneg * torch.ones_like(out).to(device))
        loss = criterion(mask_out.transpose(2, 1), label.to(device))
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print("end")

start


  0%|          | 0/370 [00:00<?, ?it/s]

<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


  0%|          | 0/370 [00:00<?, ?it/s]

<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


...

  0%|          | 0/370 [00:00<?, ?it/s]

<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


  0%|          | 0/370 [00:00<?, ?it/s]

<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
<ipython-input-6-495d11317178>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)


end


In [ ]:
# 학습된 GPT2 모델을 테스트 하는 코드는 다음과 같습니다.

sent='0'
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + sent + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

user > 안녕
Chatbot > 안녕하세요
user > 너를 소개해줘
Chatbot > 소개 해드립니다
user > 딥러닝
Chatbot > 사람마다 다른 것 같아요
user > 너는 사람이야?
Chatbot > 인공지능에 성별을 만드는 사람이죠
user > 그렇구나
Chatbot > 0
